In [23]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

def valoreOwnerInt(x):
    a = x.split()
    return int(int(a[0]) + int(a[2]))/2

def valoreOwnerStimato(x,y):
    a = x.split()
    return int(int(a[0]) + int(a[2]))/2 + y

def prova():
    df = pd.read_csv('fileAggiornato.csv')
    df['Estimated owners'] =  df.apply(lambda x:valoreOwnerStimato(x['Estimated owners'],x['Peak CCU']),axis=1)
    df['Gain'] = df['Estimated owners'] * df['Price']
    df = df.sort_values(by='Gain',ascending=False)
    layout = go.Layout(
        title='Ricavi dei primi x giochi',
        xaxis=dict(title='Gioco'),
        yaxis=dict(title='Guadagno'),
    )
    trace_gain = go.Bar(
        y=df['Name'].head(20),
        x=df['Gain'].head(20),
        name='Ricavo',
         orientation='h'
    )
    fig = go.Figure(data=[trace_gain],layout=layout)
    
    #fig = px.bar(df.head(20),'Name','Gain')
    
    fig.show()
    
    df = df.sort_values(by='Estimated owners',ascending=False)
    
    layout = go.Layout(
        title='Ricavi dei primi x giochi',
        xaxis=dict(title='Gioco'),
        yaxis=dict(title='Giocatori'),
    )
    trace_gain = go.Bar(
        x=df['Name'].head(20),
        y=df['Estimated owners'].head(20),
        name='Giocatori'
    )
    fig = go.Figure(data=[trace_gain],layout=layout)
    fig.show()
#prova()

def tryTree():
    x = [0.5,0.25,0.10,0.15]
    y = ['primo','secondo','terzo','quarto']
    
    d = {'val':x,'title':y}
    
    df = pd.DataFrame(d)
    print(df.head())
    fig = px.treemap(
        d,
        path=['title'],
        values= [df.get(title) for title in df['title']]
    )
    fig.update_layout(
        treemapcolorway = ['#67001f','#b2182b','#d6604d','#f4a582'], #defines the colors in the treemap
        margin = dict(t=50, l=25, r=25, b=25),
        width = 2160,
        height = 1080,    
    )
    
    fig.show()
tryTree()

    val    title
0  0.50    primo
1  0.25  secondo
2  0.10    terzo
3  0.15   quarto
